In [1]:
!pip install torch torchvision
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
# import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torch.optim as optim

As model is underfitting and adding dropout at each layer

 👍
Let’s break down **overfitting vs underfitting** with some **example numbers** (training loss/accuracy vs validation loss/accuracy).

---

# 📊 Example: Training a CNN on Image Classification

| Epoch | Training Accuracy | Validation Accuracy | Interpretation                                                           |
| ----- | ----------------- | ------------------- | ------------------------------------------------------------------------ |
| 1     | 55%               | 53%                 | Both are low → **Underfitting (not learned enough yet)**                 |
| 5     | 75%               | 72%                 | Both improving together → **Good fit**                                   |
| 10    | 95%               | 80%                 | Big gap appears → **Overfitting**                                        |
| 15    | 99%               | 78%                 | Training keeps improving but validation worsens → **Severe Overfitting** |

---

# 📉 Example with Loss Values

| Epoch | Training Loss | Validation Loss | Interpretation                                            |
| ----- | ------------- | --------------- | --------------------------------------------------------- |
| 1     | 1.8           | 1.9             | Both high → **Underfitting**                              |
| 5     | 0.8           | 0.9             | Both decreasing → **Good fit**                            |
| 10    | 0.2           | 0.7             | Validation loss stops decreasing → **Overfitting begins** |
| 15    | 0.05          | 1.0             | Training too perfect, validation bad → **Overfit model**  |

---

# 🔹 Key Takeaways

* **Underfitting** = Model too simple / not trained enough

  * High training loss
  * Low training accuracy
  * Training ≈ Validation (both bad)
* **Overfitting** = Model too complex / trained too long

  * Very low training loss
  * Very high training accuracy
  * Validation much worse than training
* **Good Fit** = Training and validation both improve and stay close.

---


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
dropout_value = 0.1
class ciferNet(nn.Module):
    def __init__(self):
        super(ciferNet, self).__init__()

        # Conv Block 1
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(dropout_value),
        )

        # Conv Block 2
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(dropout_value),
        )

        # Conv Block 3
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Dropout(dropout_value),
        )

        # Conv Block 4 → Depthwise separable convolution
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1, groups=128, bias=False),  # Depthwise conv
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(1, 1), bias=False),  # Pointwise conv
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Dropout(dropout_value),
        )

        # GAP and classifier
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Conv2d(in_channels=256, out_channels=10, kernel_size=(1, 1), bias=False)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.gap(x)
        x = self.classifier(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x, dim=-1)


In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = ciferNet().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
            Conv2d-4           [-1, 32, 16, 16]           4,608
              ReLU-5           [-1, 32, 16, 16]               0
       BatchNorm2d-6           [-1, 32, 16, 16]              64
           Dropout-7           [-1, 32, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          18,432
              ReLU-9           [-1, 64, 16, 16]               0
      BatchNorm2d-10           [-1, 64, 16, 16]             128
           Conv2d-11             [-1, 64, 8, 8]          36,864
             ReLU-12             [-1, 64, 8, 8]               0
      BatchNorm2d-13             [-1, 64, 8, 8]             128
          Dropout-14             [

In [4]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4741, 0.4727, 0.4733),(0.2521, 0.2520, 0.2506),)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4741, 0.4727, 0.4733),(0.2521, 0.2520, 0.2506),)
                                       ])

In [5]:
train = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

100%|██████████| 170M/170M [00:03<00:00, 43.2MB/s]


In [6]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
train_dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
test_dataloader_args = dict(shuffle=False, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=False, batch_size=64)


# train dataloader
train_loader = torch.utils.data.DataLoader(train, **train_dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **test_dataloader_args)

CUDA Available? True


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [7]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss.item()) # Added .item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss) # Added .item()

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [8]:
from torch.optim.lr_scheduler import StepLR

model =  ciferNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# scheduler = StepLR(optimizer, step_size=6, gamma=0.1)


EPOCHS = 8
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    # scheduler.step()
    test(model, device, test_loader)

EPOCH: 0


Loss=1.2774584293365479 Batch_id=390 Accuracy=44.61: 100%|██████████| 391/391 [00:18<00:00, 20.72it/s]



Test set: Average loss: 1.3091, Accuracy: 5340/10000 (53.40%)

EPOCH: 1


Loss=1.171795129776001 Batch_id=390 Accuracy=60.47: 100%|██████████| 391/391 [00:15<00:00, 25.29it/s]



Test set: Average loss: 1.0233, Accuracy: 6336/10000 (63.36%)

EPOCH: 2


...


Test set: Average loss: 0.7321, Accuracy: 7460/10000 (74.60%)

EPOCH: 6


Loss=0.4946005940437317 Batch_id=390 Accuracy=79.12: 100%|██████████| 391/391 [00:13<00:00, 28.05it/s]



Test set: Average loss: 0.6745, Accuracy: 7640/10000 (76.40%)

EPOCH: 7


Loss=0.4977283477783203 Batch_id=390 Accuracy=80.69: 100%|██████████| 391/391 [00:14<00:00, 27.66it/s]



Test set: Average loss: 0.6446, Accuracy: 7776/10000 (77.76%)

